# Revisão do artigo “Model-independent quantum phases classifier”

## Modelos Físicos

Foram usados os hamiltonianos a seguir

- cadeias XXZ com anisotropia de ion uniaxial ($H_1$), dado por:

$$\mathcal{H_1} = \sum^N_{l=1}[J(S_l^xS_{l+1}^x + S^y_lS^y_{l+1}) + J_zS_l^zS_{l+1}^z] + D \sum^N_{l=1}S_l^{z2} $$

Onde $S_l$ é o operador spin-1 agindo sobre o sítio $l$ de uma cadeia de $N$ sítios. D representa a anisotropia de ion único e $J=1, J_z$ são os acoplamentos entre os spins. O diagrama de fase desse sistema contém: Haldane, Large D, XY1, XY2, Ferromagnético e Néel.

- cadeia de vínculo alternante XXZ ($H_2$). O Hamiltoniano é dado por: 

$$\mathcal{H_2} = \sum^N_{l=1}\left[1-\delta(-1)^l\right](S_l^xS_{l+1}^x + S^y_lS^y_{l+1} + \Delta S_l^zS_{l+1}^z) $$

Aqui, $\Delta$ representa a força a anisotropia de Ising da interação spin órbita no material magnético. Já  o termo $\delta$ representa o vínculo alternante que descreve dimerização.

Tal sistema apresenta fase ferromagnéica, XX1, Néel, Haldane e Dimerizada.

- Cadeia biquadrática linear ($H_3$), dada por

$$\mathcal{H_2} = \sum^N_{l=1}[\cos \theta (S_l S_{l+1}) + \sin\theta (S_l S_{l+1})]$$

com $\theta \in [0,2\pi)$ representa o vínculo entre vizinhos. Esse modelo apresenta Haldane, Trimerizado, Ferromagnético e Dimerizado.

### Implementação do modelos

Os modelos foram implementados no arquivo `hamiltonians.py`, usando POO. Os modelos são construídos baseados em uma classe `Hamiltonian`.

In [1]:
import hamiltonians

### Parâmetros dos modelos:

Os parâmetros utilizados no modelo estão a seguir

In [2]:
import numpy as np

In [3]:
# H1 parameters
H_1_parameters = np.arange(-4, 4, 0.1)
Jzs, Ds = H_1_parameters, H_1_parameters

# H2 parameters
Deltas = np.arange(0,1, 0.005)
deltas = np.arange(-1.5,2.5, 0.0125)

# H3 parameters
thetas = np.arange(0,2*np.pi,0.001*np.pi)


## Correladores

Os modelos foram implementados no arquivo `correlators.py`, usando POO. A ideia de deixar separado do hamiltoniano é para não precisarmos inicializar os correladores caso não queiramos mudar o número de spin.

In [4]:
import correlators

## Coleta de dados

### H1

In [ ]:
N = 8
n_sup = int(N/2+1)

with open(f"data/H1/N={N}.csv", 'w') as fh:
    fh.write("Jz, D, " + ", ".join([f"S1S{i}{j}" for i in range(1, n_sup+1) for j in ['x', 'y', 'z']]) + ", "+
    ", ".join([f"prodSi{j}" for j in ['x', 'y', 'z']]))

    corr = correlators.Correlators(N)

    for Jz in Jzs:
        for D in Ds:
            data = [Jz, D]
            H1 = hamiltonians.XXZUniaxialSingleIonAnisotropy(N, Jz, D)
            gstate = H1.gstate 
            for i in range(n_sup):
                data.append(np.real(np.conj(gstate).T @ corr.S1Six(i) @ gstate))
                data.append(np.real(np.conj(gstate).T @ corr.S1Siy(i) @ gstate))
                data.append(np.real(np.conj(gstate).T @ corr.S1Siz(i) @ gstate))
            
            data.append(np.real(np.conj(gstate).T @ corr.prodSix @ gstate))
            data.append(np.real(np.conj(gstate).T @ corr.prodSiy @ gstate))
            data.append(np.real(np.conj(gstate).T @ corr.prodSiz @ gstate))

            fh.write("\n"+", ".join([str(i) for i in data]))